In [2]:
import pandas as pd
df = pd.read_csv('compustat.csv')
df.head()

C:\Users\shiho\AppData\Local\Temp\ipykernel_25956\681247003.py:2: DtypeWarning: Columns (10,12,13,17,18,20,25,26,30,32,33,34,35,647,648,649,650,654,656,664,668,669,674,675,678,679) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('compustat.csv')


,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,1004,2000-02-29,1999,3.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01
1,1004,2000-05-31,1999,4.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01
2,1004,2000-08-31,2000,1.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01
3,1004,2000-11-30,2000,2.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01
4,1004,2001-02-28,2000,3.0,5,INDL,C,D,STD,AIR,...,1.0,5080,110.0,925.0,B,IL,0.0,www.aarcorp.com,NaN,1988-01-01


In [3]:
df_with_ratios = pd.DataFrame(df[['datadate', 'gvkey', 'fqtr', 'fyr', 'tic', 'sic']])
df_with_ratios.isna().sum()

datadate      0
gvkey         0
fqtr        346
fyr           0
tic         694
sic           0
dtype: int64

Looks like gvkey would be a better primary key than ticker since it has no nans in this dataset. We also need to inspect financial quarter

In [4]:
df_with_ratios[df['fqtr'].isna()]

,datadate,gvkey,fqtr,fyr,tic,sic
2746,2014-03-31,1224,NaN,12,EGN1,4924
2747,2014-06-30,1224,NaN,12,EGN1,4924
12300,2014-11-30,1951,NaN,8,PRSNQ,3845
31199,2014-12-31,3424,NaN,9,BH,5812
35114,2014-12-31,3806,NaN,9,DWSN,1382
...,...,...,...,...,...,...
1050126,2013-06-30,196766,NaN,3,ALGDF,1040
1050128,2013-09-30,196766,NaN,3,ALGDF,1040
1074260,2015-06-30,317427,NaN,3,NOMD,2030
1074262,2015-09-30,317427,NaN,3,NOMD,2030


No clear pattern in financial quarters with nan values. Just use date value since there is no nans there

In [5]:
df_with_ratios.drop(['fqtr', 'fyr'], axis=1, inplace=True)
df_with_ratios.head()

,datadate,gvkey,tic,sic
0,2000-02-29,1004,AIR,5080
1,2000-05-31,1004,AIR,5080
2,2000-08-31,1004,AIR,5080
3,2000-11-30,1004,AIR,5080
4,2001-02-28,1004,AIR,5080


In [6]:
df['average_asset'] = (df['atq'] + df['atq'].shift(1)) / 2
df['average_inventory'] = (df['invtq'] + df['invtq'].shift(1)) / 2
df['average_receivables'] = (df['rectq'] + df['rectq'].shift(1)) / 2
df['average_workingcapital'] = (df['wcapq'] + df['wcapq'].shift(1)) / 2
df['cogs'] = df['cogsq'] / (1e-6 + df['cogsy'])

In [7]:
df_with_ratios['current_ratio'] = df['actq'] / (1e-6 + df['lctq'])
df_with_ratios['quick_ratio'] = (df['actq'] - df['invtq']) / (1e-6 + df['lctq']) 
df_with_ratios['cash_ratio'] = df['chechy'] / (1e-6 + df['lctq'] ) 
df_with_ratios['net_working_capital'] = df['actq'] - df['lctq']
df_with_ratios['debt_ratio'] = (df['dlttq'] + df['dlcq']) / (1e-6 + df['atq'])
df_with_ratios['debt_to_equity_ratio'] = (df['dlttq'] + df['dlcq']) / (1e-6 + df['ceqq']) 
df_with_ratios['equity_ratio'] = df['ceqq'] / (1e-6 + df['atq'])
df_with_ratios['cashflow_to_debt_ratio'] = df['oancfy'] / (1e-6 + (df['dlttq'] + df['dlcq']))
df_with_ratios['net_profit_margin'] = 100 * (df['niq'] / (1e-6 + df['revtq']))
df_with_ratios['return_on_assets'] = 100 * (df['niq'] / (1e-6 + df['atq']))
df_with_ratios['asset_turnover'] = df['revtq'] / (1e-6 + df['average_asset'])
df_with_ratios['inventory_turnover'] = df['cogs'] / (1e-6 + df['average_inventory'])
df_with_ratios['days_in_inventory'] = 365 / (1e-6 + (df['cogs'] / (1e-6 + df['average_inventory'])))
df_with_ratios['receivables_turnover'] = df['revtq'] / (1e-6 + df['average_inventory'])
df_with_ratios['day_sales_outstanding'] = 365 / (1e-6 + (df['revtq'] / (1e-6 + df['average_inventory'])))
df_with_ratios['working_capital_turnover'] = df['revtq'] / (1e-6 + df['average_workingcapital'])
df_with_ratios['price_to_earnings'] = df['prccq'] / (1e-6 + df['epsfxq'])
df_with_ratios['dividend_payout_ratio'] = df['dvpspq'] / (1e-6 + df['epsfxq'])
df_with_ratios['retention_ratio'] = 1 - (df['dvpspq'] / (1e-6 + df['epsfxq']))
df_with_ratios['gross_margin_ratio'] = 100 * ((df['revtq'] - df['cogsq']) / (1e-6 + df['revtq']))
df_with_ratios['operating_profit_margin'] = 100 * ((df['revtq'] - df['cogsq']- df['xoprq']) / (1e-6 + df['revtq']))
df_with_ratios['ebitda_margin'] = 100 * ((df['revtq'] - df['cogsq']- df['xoprq'] + df['dpq']) / (1e-6 + df['revtq']))
df_with_ratios['debt_service_coverage_ratio'] = (df['revtq'] - df['cogsq']- df['xoprq']) / (1e-6 + (df['dlttq'] + df['dlcq']))
df_with_ratios['interest_coverage_ratio'] = df['oibdpq'] / (1e-6 + df['xintq'])

In [8]:
df_with_ratios.head(5)

,datadate,gvkey,tic,sic,current_ratio,quick_ratio,cash_ratio,net_working_capital,debt_ratio,debt_to_equity_ratio,...,day_sales_outstanding,working_capital_turnover,price_to_earnings,dividend_payout_ratio,retention_ratio,gross_margin_ratio,operating_profit_margin,ebitda_margin,debt_service_coverage_ratio,interest_coverage_ratio
0,2000-02-29,1004,AIR,5080,2.981531,1.176928,-0.021609,350.202,0.273288,0.601468,...,NaN,NaN,59.374852,0.212499,0.787501,18.234795,-72.491563,-70.807950,-0.958372,4.223950
1,2000-05-31,1004,AIR,5080,3.120983,1.069792,-0.042786,347.451,0.279030,0.608989,...,531.050804,0.645246,154.164954,0.944434,0.055566,18.944015,-75.280235,-73.157424,-0.819497,2.287926
2,2000-08-31,1004,AIR,5080,3.032279,0.989191,-0.003399,341.996,0.305692,0.673592,...,513.170982,0.701345,93.749219,0.708327,0.291673,16.176118,-77.799561,-75.858047,-0.823112,2.432364
3,2000-11-30,1004,AIR,5080,2.718261,0.897575,0.009653,325.382,0.298354,0.675752,...,594.638929,0.633329,64.843345,0.531247,0.468753,18.857738,-73.583646,-71.446755,-0.674333,2.793634
4,2001-02-28,1004,AIR,5080,3.136912,1.025058,-0.002824,358.685,0.304446,0.666264,...,637.844464,0.584946,67.999660,0.424998,0.575002,20.259808,-71.119752,-68.726602,-0.619269,3.174489


In [9]:
print(f"length of dataset: {len(df_with_ratios)}")
print(f"shape of dataset: {df_with_ratios.shape}")

length of dataset: 1075571
shape of dataset: (1075571, 28)


In [10]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan'] = (df_with_ratios.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown

,pct_nan
datadate,0.000000
gvkey,0.000000
tic,0.064524
sic,0.000000
current_ratio,37.207679
quick_ratio,38.331268
cash_ratio,38.432609
net_working_capital,37.207679
debt_ratio,28.283210
debt_to_equity_ratio,28.558598


At this point our dataset should have columns with at most 20-30% nan values. we shall now inpute them using sector median. 
Let us first try Methodology 1: 
1. map gvkey to sector
2. Eg. if company X in sector A on date 2000-01-01 has a missing value for total income, we inpute it using the median of sector A in 2000-01-01. 

This assumes that on a particular date, a companies fundamentals can be represented by its sector median on that date. 
Sector will be represented by SIC code. (I used all 4 digits for now)

First let us convert datadate to year and quarter representation

In [11]:
df_with_ratios['datadate'] = pd.to_datetime(df_with_ratios['datadate'])
df_with_ratios['year'] = df_with_ratios['datadate'].dt.year
df_with_ratios['quarter'] = df_with_ratios['datadate'].dt.quarter
df_with_ratios.head()

,datadate,gvkey,tic,sic,current_ratio,quick_ratio,cash_ratio,net_working_capital,debt_ratio,debt_to_equity_ratio,...,price_to_earnings,dividend_payout_ratio,retention_ratio,gross_margin_ratio,operating_profit_margin,ebitda_margin,debt_service_coverage_ratio,interest_coverage_ratio,year,quarter
0,2000-02-29,1004,AIR,5080,2.981531,1.176928,-0.021609,350.202,0.273288,0.601468,...,59.374852,0.212499,0.787501,18.234795,-72.491563,-70.807950,-0.958372,4.223950,2000,1
1,2000-05-31,1004,AIR,5080,3.120983,1.069792,-0.042786,347.451,0.279030,0.608989,...,154.164954,0.944434,0.055566,18.944015,-75.280235,-73.157424,-0.819497,2.287926,2000,2
2,2000-08-31,1004,AIR,5080,3.032279,0.989191,-0.003399,341.996,0.305692,0.673592,...,93.749219,0.708327,0.291673,16.176118,-77.799561,-75.858047,-0.823112,2.432364,2000,3
3,2000-11-30,1004,AIR,5080,2.718261,0.897575,0.009653,325.382,0.298354,0.675752,...,64.843345,0.531247,0.468753,18.857738,-73.583646,-71.446755,-0.674333,2.793634,2000,4
4,2001-02-28,1004,AIR,5080,3.136912,1.025058,-0.002824,358.685,0.304446,0.666264,...,67.999660,0.424998,0.575002,20.259808,-71.119752,-68.726602,-0.619269,3.174489,2001,1


In [12]:
df_drop_duplicates = df_with_ratios.drop_duplicates(subset=['gvkey', 'year', 'quarter'])
len(df_with_ratios) - len(df_drop_duplicates)

1402

We then drop rows where there are duplicate year, quarters and gvkey, since the original dataset is monthly data. We only drop 1402 rows, so not significant. 

In [13]:
df_with_ratios = df_drop_duplicates

Now we find all the gvkeys where there is no data at all and drop them from our dataframe. From this operation we remove 6813 gvkeys from a total of 

In [14]:
df_drop_gvkey = df_with_ratios.drop(['sic',  'tic', 'year', 'quarter', 'datadate'], axis=1)
df_drop_gvkey = df_drop_gvkey.groupby('gvkey').apply(lambda group: group.iloc[:, 1:].isna().all().all())
to_drop = df_drop_gvkey[df_drop_gvkey == True]
gvkeys_to_drop = list(to_drop.index)

In [15]:
original_number_of_gvkeys = len(df_with_ratios['gvkey'].unique())
print(f"Dropped {len(gvkeys_to_drop) * 100 / original_number_of_gvkeys}% of stocks")

Dropped 22.53381394887397% of stocks


In [16]:
df_with_ratios_dropped_gvkey = df_with_ratios[~df_with_ratios['gvkey'].isin(gvkeys_to_drop)]
print(f"number of unique gvkeys now: {len(df_with_ratios_dropped_gvkey['gvkey'].unique())}")
print(f"% of total rows lost: {(len(df_with_ratios) - len(df_with_ratios_dropped_gvkey)) * 100 / len(df_with_ratios)  }%")

number of unique gvkeys now: 23425
% of total rows lost: 17.750651899282143%


In [17]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan'] = (df_with_ratios_dropped_gvkey.isna().sum() / len(df_with_ratios_dropped_gvkey)) * 100
nan_breakdown

,pct_nan
datadate,0.000000
gvkey,0.000000
tic,0.027957
sic,0.000000
current_ratio,23.658937
quick_ratio,25.026344
cash_ratio,25.122326
net_working_capital,23.658937
debt_ratio,12.797327
debt_to_equity_ratio,13.132246


We can also see that the % of nans has decreased significantly across columns after dropping these gvkeys

In [18]:
test = df_with_ratios_dropped_gvkey
test = test[test.isna().any(axis=1)]
test[test['tic'] == 'AFAP']

,datadate,gvkey,tic,sic,current_ratio,quick_ratio,cash_ratio,net_working_capital,debt_ratio,debt_to_equity_ratio,...,price_to_earnings,dividend_payout_ratio,retention_ratio,gross_margin_ratio,operating_profit_margin,ebitda_margin,debt_service_coverage_ratio,interest_coverage_ratio,year,quarter
156,2000-03-31,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,84.745715,0.282486,0.717514,NaN,NaN,NaN,NaN,NaN,2000,1
157,2000-06-30,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,49.848928,0.151057,0.848943,NaN,NaN,NaN,NaN,NaN,2000,2
158,2000-09-30,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,3
159,2000-12-31,1019,AFAP,7380,1.640074,1.550835,0.116883,3.450,0.062365,0.135467,...,34.014864,0.092937,0.907063,-17.483221,-195.148600,-185.426645,-22.792820,24.407962,2000,4
160,2001-03-31,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,75.100371,0.200803,0.799197,NaN,NaN,NaN,NaN,NaN,2001,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2019-12-31,1019,AFAP,7380,1.405545,1.159588,0.031515,6.846,0.054954,0.157569,...,13.225568,0.036738,0.963262,32.806181,-60.904101,-58.519882,-27.607186,44.666285,2019,4
236,2020-03-31,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,1
237,2020-06-30,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,2
238,2020-09-30,1019,AFAP,7380,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,3


Here we can see that there are still certain stocks where there is data, but many missing values. for example for ticker AFAP, we have data missing from 2003-2016. Such tickers should be dropped as well

In [19]:
cols_df = df_with_ratios_dropped_gvkey.drop(['gvkey', 'sic',  'tic', 'year', 'quarter', 'datadate'], axis=1)
all_na_rows = cols_df[cols_df.isna().all(axis=1)]
df_with_nans = df_with_ratios_dropped_gvkey.loc[all_na_rows.index]
df_with_nans_counts = df_with_nans.groupby('gvkey').size()
gvkey_with_bad_data = df_with_nans_counts[df_with_nans_counts > 10]

In [20]:
print(len(df_with_ratios_dropped_gvkey['gvkey'].unique()))
print(100*len(gvkey_with_bad_data) / len(df_with_ratios_dropped_gvkey['gvkey'].unique()))

23425
2.992529348986126


We can decide here the threshold for which we want to drop our gvkeys. How many nan rows can we allow a gvkey to have? For now i set the value at less than 10. This causes us to drop 671 gvkeys, or 2.86% of gvkeys. 

In [21]:
df_with_ratios_dropped_gvkey_v2 = df_with_ratios_dropped_gvkey[~df_with_ratios_dropped_gvkey['gvkey'].isin(gvkey_with_bad_data.index)]
len(df_with_ratios_dropped_gvkey_v2)

843144

In [22]:
print(f"% of total rows lost: {(len(df_with_ratios) - len(df_with_ratios_dropped_gvkey_v2)) * 100 / len(df_with_ratios)  }%")

% of total rows lost: 21.50732333552728%


In [23]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan'] = (df_with_ratios_dropped_gvkey_v2.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown

,pct_nan
datadate,0.000000
gvkey,0.000000
tic,0.022995
sic,0.000000
current_ratio,17.353787
quick_ratio,18.446632
cash_ratio,18.491317
net_working_capital,17.353787
debt_ratio,8.762495
debt_to_equity_ratio,9.018972


Our % of rows dropped increased from 17% to 21%. We can reduce this perentage by increasing the tolerance of how many nan rows a stock can have. This is balancing data quantity vs quality. 

In [24]:
# dealing with inf value
inf_breakdown = pd.DataFrame()
inf_breakdown['pct_inf'] = (100 * df_with_ratios_dropped_gvkey_v2.isin([float('inf'), float('-inf')]).sum()) / len(df_with_ratios_dropped_gvkey_v2)
inf_breakdown

,pct_inf
datadate,0.0
gvkey,0.0
tic,0.0
sic,0.0
current_ratio,0.0
quick_ratio,0.0
cash_ratio,0.0
net_working_capital,0.0
debt_ratio,0.0
debt_to_equity_ratio,0.0


In [25]:
df_with_ratios_dropped_gvkey_v2['cashflow_to_debt_ratio'].quantile(0.1)

-66000.0

In [26]:
df_with_ratios_dropped_gvkey_v2

,datadate,gvkey,tic,sic,current_ratio,quick_ratio,cash_ratio,net_working_capital,debt_ratio,debt_to_equity_ratio,...,price_to_earnings,dividend_payout_ratio,retention_ratio,gross_margin_ratio,operating_profit_margin,ebitda_margin,debt_service_coverage_ratio,interest_coverage_ratio,year,quarter
0,2000-02-29,1004,AIR,5080,2.981531,1.176928,-0.021609,350.202,0.273288,0.601468,...,59.374852,0.212499,0.787501,18.234795,-72.491563,-70.807950,-0.958372,4.223950,2000,1
1,2000-05-31,1004,AIR,5080,3.120983,1.069792,-0.042786,347.451,0.279030,0.608989,...,154.164954,0.944434,0.055566,18.944015,-75.280235,-73.157424,-0.819497,2.287926,2000,2
2,2000-08-31,1004,AIR,5080,3.032279,0.989191,-0.003399,341.996,0.305692,0.673592,...,93.749219,0.708327,0.291673,16.176118,-77.799561,-75.858047,-0.823112,2.432364,2000,3
3,2000-11-30,1004,AIR,5080,2.718261,0.897575,0.009653,325.382,0.298354,0.675752,...,64.843345,0.531247,0.468753,18.857738,-73.583646,-71.446755,-0.674333,2.793634,2000,4
4,2001-02-28,1004,AIR,5080,3.136912,1.025058,-0.002824,358.685,0.304446,0.666264,...,67.999660,0.424998,0.575002,20.259808,-71.119752,-68.726602,-0.619269,3.174489,2001,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075564,2021-12-31,353945,ACLLY,3621,1.339565,0.865039,0.211009,110.740,0.179542,0.426455,...,NaN,NaN,NaN,50.122808,-22.580129,-19.472124,-1.386667,149.741008,2021,4
1075565,2022-03-31,353945,ACLLY,3621,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,1
1075566,2022-06-30,353945,ACLLY,3621,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,2
1075567,2022-09-30,353945,ACLLY,3621,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,3


In [27]:
sector_date_median = df_with_ratios_dropped_gvkey_v2.groupby(['sic', 'year', 'quarter']).median()
sector_date_median

TypeError: agg function failed [how->median,dtype->object]

In [ ]:
cols_to_check = [col for col in df_with_ratios_dropped_gvkey_v2.columns if col not in ['datadate', 'year', 'quarter', 'sic', 'gvkey', 'tic']]
cols_to_check

['current_ratio',
 'quick_ratio',
 'cash_ratio',
 'net_working_capital',
 'debt_ratio',
 'debt_to_equity_ratio',
 'equity_ratio',
 'cashflow_to_debt_ratio',
 'net_profit_margin',
 'return_on_assets',
 'asset_turnover',
 'inventory_turnover',
 'days_in_inventory',
 'receivables_turnover',
 'day_sales_outstanding',
 'working_capital_turnover',
 'price_to_earnings',
 'dividend_payout_ratio',
 'retention_ratio',
 'gross_margin_ratio',
 'operating_profit_margin',
 'ebitda_margin',
 'debt_service_coverage_ratio',
 'interest_coverage_ratio']

In [ ]:
import numpy as np
def fill_with_median(x):
    sic = x['sic']
    year = x['year']
    quarter = x['quarter']
    for col in cols_to_check: 
        if np.isnan(x[col]): 
            median = sector_date_median[sector_date_median.index == (sic,year,quarter)][col].iloc[0]
            x[col] = median
    return x

In [ ]:
df_inputed1 = df_with_ratios_dropped_gvkey_v2[:50000].apply(fill_with_median, axis=1)
df_inputed2 = df_with_ratios_dropped_gvkey_v2[50000:100000].apply(fill_with_median, axis=1)
df_inputed3 = df_with_ratios_dropped_gvkey_v2[100000:150000].apply(fill_with_median, axis=1)

In [ ]:
df_inputed4 = df_with_ratios_dropped_gvkey_v2[150000:200000].apply(fill_with_median, axis=1)
df_inputed5 = df_with_ratios_dropped_gvkey_v2[200000:250000].apply(fill_with_median, axis=1)
df_inputed6 = df_with_ratios_dropped_gvkey_v2[250000:300000].apply(fill_with_median, axis=1)

In [ ]:
df_inputed7 = df_with_ratios_dropped_gvkey_v2[300000:350000].apply(fill_with_median, axis=1)
df_inputed8 = df_with_ratios_dropped_gvkey_v2[350000:400000].apply(fill_with_median, axis=1)
df_inputed9 = df_with_ratios_dropped_gvkey_v2[400000:450000].apply(fill_with_median, axis=1)

In [ ]:
combined = pd.DataFrame()
for df in [
    df_inputed1,
    df_inputed2,
        df_inputed3,
    df_inputed4,
        df_inputed5,
    df_inputed6,
        df_inputed7,
    df_inputed8,
    df_inputed9,
]:
    combined = pd.concat([combined, df])
    
    

In [ ]:
df_inputed10 = df_with_ratios_dropped_gvkey_v2[450000:500000].apply(fill_with_median, axis=1)
df_inputed11 = df_with_ratios_dropped_gvkey_v2[500000:550000].apply(fill_with_median, axis=1)
df_inputed12 = df_with_ratios_dropped_gvkey_v2[550000:600000].apply(fill_with_median, axis=1)

In [ ]:
df_inputed13 = df_with_ratios_dropped_gvkey_v2[600000:650000].apply(fill_with_median, axis=1)
df_inputed14 = df_with_ratios_dropped_gvkey_v2[650000:700000].apply(fill_with_median, axis=1)
df_inputed15 = df_with_ratios_dropped_gvkey_v2[700000:750000].apply(fill_with_median, axis=1)

In [ ]:

df_inputed16 = df_with_ratios_dropped_gvkey_v2[750000:800000].apply(fill_with_median, axis=1)
df_inputed17 = df_with_ratios_dropped_gvkey_v2[800000:850000].apply(fill_with_median, axis=1)

In [ ]:
for df in [
    df_inputed10,
    df_inputed11,
        df_inputed12,
    df_inputed13,
        df_inputed14,
    df_inputed15,
        df_inputed16,
    df_inputed17,
]:
    combined = pd.concat([combined, df])

In [ ]:
combined.to_pickle('./combined_final_22_oct.pkl')

In [ ]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan_before_sector_inpute'] = (df_with_ratios_dropped_gvkey_v2.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown['pct_nan_after_sector_inpute'] = (combined.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown

,pct_nan_before_sector_inpute,pct_nan_after_sector_inpute
gvkey,0.000000,0.000000
sic,0.000000,0.000000
current_ratio,17.353787,6.000173
quick_ratio,18.446632,6.037039
cash_ratio,18.491317,6.110305
net_working_capital,17.353787,6.000173
debt_ratio,8.762495,0.037145
debt_to_equity_ratio,9.018972,0.038076
equity_ratio,5.044365,0.018712
cashflow_to_debt_ratio,11.673117,0.237579


In [ ]:
inputed_data_ffill = combined.ffill(limit=4)

In [ ]:
nan_breakdown = pd.DataFrame()
nan_breakdown['pct_nan_before_sector_inpute'] = (df_with_ratios_dropped_gvkey_v2.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown['pct_nan_after_sector_inpute'] = (combined.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown['pct_nan_after_sector_inpute_and_ffill'] = (inputed_data_ffill.isna().sum() / len(df_with_ratios)) * 100
nan_breakdown

,pct_nan_before_sector_inpute,pct_nan_after_sector_inpute,pct_nan_after_sector_inpute_and_ffill
gvkey,0.000000,0.000000,0.000000
sic,0.000000,0.000000,0.000000
current_ratio,17.353787,6.000173,4.738267
quick_ratio,18.446632,6.037039,4.740222
cash_ratio,18.491317,6.110305,4.600021
net_working_capital,17.353787,6.000173,4.738267
debt_ratio,8.762495,0.037145,0.001490
debt_to_equity_ratio,9.018972,0.038076,0.001490
equity_ratio,5.044365,0.018712,0.000000
cashflow_to_debt_ratio,11.673117,0.237579,0.004282


In [ ]:
print(len(df_with_ratios))
print(len(inputed_data_ffill))

1074169
843144


In [ ]:
inputed_data_ffill_dropna = inputed_data_ffill.dropna()

In [ ]:
len(inputed_data_ffill_dropna)

784990

In [ ]:
print(len(df_with_ratios['gvkey'].unique()))
print(len(inputed_data_ffill_dropna['gvkey'].unique()))

30239
22638


In [ ]:
inputed_data_ffill_dropna.to_pickle('compustat_imputed.pkl')

In [ ]:
inputed_data_ffill_dropna

,gvkey,sic,current_ratio,quick_ratio,cash_ratio,net_working_capital,debt_ratio,debt_to_equity_ratio,equity_ratio,cashflow_to_debt_ratio,...,price_to_earnings,dividend_payout_ratio,retention_ratio,gross_margin_ratio,operating_profit_margin,ebitda_margin,debt_service_coverage_ratio,interest_coverage_ratio,year,quarter
0,1004.0,5080.0,2.981531,1.176928,-0.021609,350.202,0.273288,0.601468,0.454368,0.027511,...,59.374852,0.212499,0.787501,18.234795,-72.491563,-70.807950,-0.958372,4.223950,2000.0,1.0
1,1004.0,5080.0,3.120983,1.069792,-0.042786,347.451,0.279030,0.608989,0.458186,0.048612,...,154.164954,0.944434,0.055566,18.944015,-75.280235,-73.157424,-0.819497,2.287926,2000.0,2.0
2,1004.0,5080.0,3.032279,0.989191,-0.003399,341.996,0.305692,0.673592,0.453824,-0.033884,...,93.749219,0.708327,0.291673,16.176118,-77.799561,-75.858047,-0.823112,2.432364,2000.0,3.0
3,1004.0,5080.0,2.718261,0.897575,0.009653,325.382,0.298354,0.675752,0.441514,0.044014,...,64.843345,0.531247,0.468753,18.857738,-73.583646,-71.446755,-0.674333,2.793634,2000.0,4.0
4,1004.0,5080.0,3.136912,1.025058,-0.002824,358.685,0.304446,0.666264,0.456946,0.008334,...,67.999660,0.424998,0.575002,20.259808,-71.119752,-68.726602,-0.619269,3.174489,2001.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075564,353945.0,3621.0,1.339565,0.865039,0.211009,110.740,0.179542,0.426455,0.421011,1.325553,...,60.623448,0.000000,1.000000,50.122808,-22.580129,-19.472124,-1.386667,149.741008,2021.0,4.0
1075565,353945.0,3621.0,2.085575,1.309577,-0.045523,115.160,0.208154,0.138676,0.571360,-0.083261,...,157.167464,0.000000,1.000000,31.883879,-61.215036,-57.122944,-0.527729,15.635813,2022.0,1.0
1075566,353945.0,3621.0,2.379581,1.178016,-0.010057,128.886,0.211288,0.071914,0.562983,-0.020901,...,-16.187551,0.000000,1.000000,27.749175,-86.195581,-83.177332,-0.616682,-2.972347,2022.0,2.0
1075567,353945.0,3621.0,2.517124,1.392924,-0.002727,222.247,0.220292,0.308750,0.528746,-0.031245,...,-12.757195,-0.000000,1.000000,29.844044,-72.769987,-68.926603,-0.508082,2.647169,2022.0,3.0
